# Packages, Libraries, and Constants
- Different packages, Libraries

- Different constants and parameters

## Path to the datasets

1. Speech Intent Classification (SIC) Dataset
 - `emabega` - `ddyo` - `unknown` - `kkono`  - `yimirira` - `mu maaso` 

2. Datasets
- `Augmented train data` - `Original train data` - `Test data`


In [ ]:
from packages.utils import *

## Load the mfcc json files

In [52]:
from packages.mfcc import load_data

TRAIN_DATA_PATH = "json/mfcc_train_data.json"
TEST_DATA_PATH = "json/mfcc_test_data.json"

train_data = load_data(TRAIN_DATA_PATH)
test_data = load_data(TEST_DATA_PATH)

In [53]:
# Convert lists to numpy arrays
X_train = np.array(train_data["mfcc"])
y_train = np.array(train_data["labels"])
X_test = np.array(test_data["mfcc"])
y_test = np.array(test_data["labels"])

# Split training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

## Model

In [54]:
from packages.model import model

# Input shape
input_shape = (X_train.shape[1], X_train.shape[2], 1)
input_shape

model = model(input_shape)
model.summary()

### Compile and train model

In [ ]:
# Compile model
learning_rate = 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=64, epochs=50)

### Accuracy

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.2f}')

In [ ]:
# Assuming you have the test data prepared as X_test and y_test
# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Calculate accuracy, precision, and F1 score
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

# Print classification report
class_report = classification_report(y_test, y_pred_classes)
print("Classification Report:\n", class_report)

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='g', xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


### Plot a history

In [ ]:
# Function to plot the training history
from packages.model import plot_training_history

# Plot the training history
plot_training_history(history)

### Save a model

In [ ]:
model.save("audio_classification_model.h5")

In [ ]:
def predict(file_path, model):
    # Load the audio file
    signal, sr = librosa.load(file_path, sr=SAMPLE_RATE)

    # Extract MFCCs
    mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13, n_fft=2048, hop_length=512)
    mfcc = mfcc.T

    # Ensure the correct shape
    mfcc = np.expand_dims(mfcc, axis=-1)
    mfcc = np.expand_dims(mfcc, axis=0)

    # Make prediction
    prediction = model.predict(mfcc)
    predicted_index = np.argmax(prediction, axis=1)
    return predicted_index

# Load the model
model = tf.keras.models.load_model('audio_classification_model.h5')

In [65]:
train_data_dir = pathlib.Path('Dataset/Train')

emabega_file_path = os.path.join(train_data_dir, 'emabega', 'emabega_001.wav')
ddyo_file_path = os.path.join(train_data_dir, 'ddyo', 'ddyo_001.wav')

In [ ]:
#  Predict
file_path = ddyo_file_path
predicted_class = predict(file_path, model)
print(f'Predicted Class: {predicted_class}')

### Save a model

In [67]:
KERAS_MODEL_PATH = "model/mfcc_model_1.keras"
model.save(KERAS_MODEL_PATH)


### Get model size

In [ ]:
from packages.model import get_model_size

# Model size
keras_model_size = get_model_size(KERAS_MODEL_PATH, 'KB')